# Indizes

Im vorherigen Kapitel wurde schon erwähnt, dass der Zugriff auf Daten durch Indizes beschleunigt werden kann. Nach einem **naiven** Ansatz, wo die Datensätze beliebig verteilt sind, muss jeder Block untersucht werden, wenn wir die Anfrage ```SELECT * FROM R``` ausführen. Eine Verbesserung davon wäre, die Tupel einer Relation zusammenhängend zu speichern. Wenn wir die Anfrage ```SELECT * FROM R WHERE a=10``` ausführen wollen, müssen jedoch alle Datensätze betrachtet werden.
<br><br>
Noch besser im Gegensatz dazu ist die Benutzung von **Indizes**. Es werden Eigenschaften von Datensätzen, wie z.B „Suchschlüssel“ (nicht zu verwechseln mit Primärschlüssel, Sekundärschlüssel, Sortierschlüssel) festgelegt. Der Suchschlüssel ist der Wert, nach welchem gesucht werden soll. Das Ziel ist unter anderem, I/O-Kosten zu minimieren und möglichst wenige Datensätze zu betrachten. Durch richtiges Verwenden von Indizes kommt es zu einem schnelleren Output der entsprechenden Tupel.

<img src="pictures/Überblick-meme.png" alt="Überblick-meme" width="500" style="background-color: white;"/>

## Indizes auf sequenziellen Dateien
### Einfachste Form eines Index
Je nachdem wie die Tupel organisiert sind, gibt es verschiedene Varianten Indizes anzulegen. Die einfachste Form davon ist, wenn eine nach unserem Suchschlüssel sortierte Datei gegeben ist. Dazu gibt es eine Indexdatei, welche  Schlüssel-Pointer Paare enthält. Jeder Schlüsselwert K ist mit einem Pointer verbunden, welcher auf den Datensatz zeigt, der den Schlüsselwert K enthält. Davon gibt es zwei Varianten, einmal den dichtbestzten und einmal den dünnbesetzten Index. Im dichtbesetzten Index gibt es für jeden Datensatz einen Eintrag im Index. Im dünnbesetzten Index, werden nur einige Datensätze im Index repräsentiert, z.B ein Eintrag pro Block.

**Sequenzielle Dateien: Index Beispiel**
<br>
In der folgenden Abbildung sind sequenzielle Daten dargestellt. Es werden jeweils zwei Tupel pro Block gespeichert und es werden insgesamt 5 Blöcke, für unsere 10 Tupel benötigt. In unserem Beispiel ist der Schlüssel eine Zahl. Häufig ist der Suchschlüssel auch der Primärschlüssel. Das Schlüsselfeld steht einfachheitshalber an erster Stelle. Um beispielsweise den Wert 70 zu finden, müssen 4 Blöcke gelesen werden. Im folgenden sehen wir, wie das mit Indizes verbessert werden kann.

<img src="pictures/Sequentielle-Dateien.png" alt="Sequentielle-Dateien" width="500" style="background-color: white;"/>

### Dichtbesetzte Indizes

Ein dichtbesetzter Index bildet sich aus einer Blocksequenz mit Schlüssel-Pointer Paaren. Jeder Schlüssel der Daten ist durch ein Paar repräsentiert. Die Datenmenge ist aber wesentlich kleiner, da im Index nur auf bestimmte Attribute gezeigt wird. Daher passt der Index womöglich in den Hauptspeicher und braucht nur einen I/O pro Zugriff. Die Sortierung der Paare entspricht der Sortierung der Daten.

**Anfragebearbeitung mit dichtbesetzten Indizes**

Es ist ein Suchschlüssel K gegeben. Die Indexdatei wird nach K durchsucht und es wird dem zugehörigen Pointer gefolgt. Der Block wird dann aus der Datendatei geladen. Wenn die Indexdatei nur wenige Blöcke hat, befindet sie sich schon im Hauptspeicher. Andernfalls wird die binäre Suche angewendet um K zu finden.

     
**Beispiel**: Wir haben 1.000.000 Tupel gegeben. Ein Block speichert 4096 Byte = 10 Tupel. Die Gesamtgröße beträgt demnach 400 MB. Zusätzlich belegt ein Schlüsselfeld je 30 Byte und ein Pointer 8 Byte, d.h. wir haben 100 Paare pro Block. Für einen dichtbesetzten Index sind 10.000 Blöcke notwendig, das sind 40 MB (vielleicht OK im Hauptspeicher).
Bei der binären Suche werden 13-14 Blocks pro Suche betrachtet (log2(10.000) ≈ 13). Es reicht wenn die wichtigsten Blöcke im Hauptspeicher sind.

<img src="pictures/Dichtbesetzte-Indizes.png" alt="Dichtbesetzte-Indizes" width="500" style="background-color: white;"/>

### Dünnbesetzte Indizes
Da wir eine sortierte Liste gegeben haben, kann auch ein dünnbesetzter Index eingesetzt werden. Hierbei ist der Schlüsselwert der kleinste Wert des referenzierten Blocks und es gibt nur einen Pointer pro Block. Der Vorteil ist weniger Speicherbedarf, jedoch erhöht sich der Suchaufwand. 
<br><br>
**Beispiel**:
Wir haben 1.000.000 Tupel gegeben. Ein Block speichert 4096 Byte = 10 Tupel. Die Gesamtgröße beträgt demnach 400 MB. Zusätzlich belegt ein Schlüsselfeld je 30 Byte und ein Pointer 8 Byte, d.h. wir haben 100 Paare pro Block. Nun gibt es 100.000 Datenblöcke und 100 Indexpaare pro Block. Demnach sind für einen dünnbesetzten Index 1.000 Blocks = 4MB notwendig, welches erheblich weniger als 400MB ist.
     
#### Anfragebearbeitung mit dünnbesetzten Indizes

1. Suche im Index größten Schlüssel, der gleich/kleiner als Suchschlüssel ist (leicht modifizierte binäre Suche)
2. Hole assoziierten Datenblock
3. Durchsuche Block nach Datensatz


**Was kann ein dünnbesetzter Index nicht?**
<br>
Mit ausschließlich einem dünnbesetzten Index kann nicht überprüft werden, ob ein bestimmter Wert vorhanden ist oder nicht. <br>
**Beispiel:** ```SELECT 'TRUE' FROM R WHERE a=10```. <br>
Mit einem dichtbesetzten Index ist das möglich. Gleiches gilt für einen Semi-Join.


<img src="pictures/Dünnbesetzte-Indizes.png" alt="Dünnbesetzte-Indizes" width="500" style="background-color: white;"/>

<img src="pictures/Mehrstufiger-Index-meme1.png" alt="Mehrstufiger-Index-meme1" width="500" style="background-color: white;"/> <img src="pictures/Mehrstufiger-Index-meme2.png" alt="Mehrstufiger-Index-meme2" width="500" style="background-color: white;"/>

### Mehrstufiger Index
Auch ein Index kann unangenehm groß sein, z.B GB groß oder sogar größer als die Datensätze selbst. Das nimmt viel Speicher ein und kostet viel I/O, auch bei binärer Suche. In diesem Fall lohnt es sich die Indexdatei auch zu indexieren. Der zweite Index macht nur als dünnbesetzten Index Sinn. Theoretisch sind auch dritte, vierte, … Ebenen möglich, in diesen Fällen ist ein B-Baum besser geeignet (dazu später).
    
**Mehrstufiger Index Beispiel**

Wir haben 1.000.000 Tupel gegeben. Ein Block speichert 4096 Byte = 10 Tupel. Die Gesamtgröße beträgt demnach 400 MB. Zusätzlich belegt ein Schlüsselfeld je 30 Byte und ein Pointer 8 Byte, d.h. wir haben 100 Paare pro Block. Nun gibt es 100.000 Datenblöcke und 100 Indexpaare pro Block. Für den Index erster Stufe sind 1.000 Blöcke = 4MB und für den Index zweiter Stufe = 40KB nötig. Der Index kann daher mit Sicherheit im Hauptspeicher verbleiben.

**Vorgehen zur Anfragebearbeitung**
1. Suche im Index zweiter Stufe den größten Schlüssel, der kleiner/gleich als Suchschlüssel ist.
2. Hole entsprechenden Block im Index erster Stufe (eventuell schon im Hauptspeicher)
3. Suche in dem Block den größten Schlüssel, der kleiner/gleich als Suchschlüssel ist.
4. Hole den entsprechenden Datenblock.
5. Suche den Datensatz (falls Index erster Stufe dünnbesetzt ist).
<br><br>

=> Für das Beispiel oben sind zusammen nur 2 I/Os nötig

<img src="pictures/Mehrstufiger-Index.png" alt="Mehrstufiger-Index" width="500" style="background-color: white;"/>

### Indizes für Nicht-eindeutige Suchschlüssel

Bisher haben wir angenommen, das unser Suchschlüssel auch ein Schlüssel ist bzw. nur maximal einmal in unserer Relation vorkommt. Jetzt betrachten wir die Indexwahl, wenn der Suchschlüssel nicht-eindeutig ist. Wir nehmen weiterhin an, dass die Relation nach unserem Suchschlüssel sortiert ist.
<br><br>
**Idee 1: Dichtbesetzter Index** 
<br>
Es gibt ein Paar im Index für jeden Datensatz. Die Anfragebearbeitung verläuft wie folgt:
- Suche erstes Paar mit K. 
- Wähle alle weiteren mit K (direkt dahinter) 
- Hole entsprechende Datensätze. 

**Idee 2: Nur ein Indexpaar pro eindeutigem Schlüsselwert K**
<br>
Der Index zeigt auf den ersten Datensatz mit K. Alle weiteren Datensätze mit K folgen direkt. Wichtig zu beachten ist hier, dass die Blöcke Pointer auf den jeweils nächsten Block haben.

<img src="pictures/Indizes-Nicht_eindeutige-Suchschlüssel.png" alt="Indizes-Nicht_eindeutige-Suchschlüssel" width="500" style="background-color: white;"/>
   

**Idee 3: Dünnbesetzter Index wie gehabt**
<br>
Der Datenwert wird jeweils am Blockanfang des Datensatzes indexiert.
Die Anfragebearbeitung ist wie folgt:
 - Suche letzten Eintrag E1 im Index, dessen Datenwert ≤ K
 - Suche von dort im Index nach vorn bis zu einem Eintrag E2 mit Datenwert < K
 - Hole alle Datenblöcke zwischen und inklusive E1 und E2.
 <br><br>
 
Beispiel: Wir suchen K = 20. Zuerst muss im "10er-Block" gesucht werden ,da womöglich auch in diesem Block eine 20 ist. Eine Rückwärtssuche ist nötig

<img src="pictures/Indizes-Nicht_eindeutige-Suchschlüssel_2.png" alt="Indizes-Nicht_eindeutige-Suchschlüssel_2" width="500" style="background-color: white;"/>

**Idee 4: Dünnbesetzter Index, aber...**
<br>
...der im Index gespeicherte Datenwert ist der kleinste neue Wert im entsprechenden Datenblock.
<br>
Die Anfragebearbeitung ist so einfacher:
- Suche im Index nach Paar mit (Datenwert = K) oder (größter Wert mit < K aber nächster Wert ist > K).
- Hole Datenblock und gegebenenfalls folgende Datenblöcke.
<br><br>

Im Beispiel unten zeigt das ⊥ ,dass nach der „30“ kein neuer kleinster Wert mehr folgt.

<img src="pictures/Indizes-Nicht_eindeutige-Suchschlüssel_3.png" alt="Indizes-Nicht_eindeutige-Suchschlüssel_3" width="500" style="background-color: white;"/>

### Änderungsoperationen

Daten ändern sich durch Insert-, Update-, und Deleteoperationen. Unsere bisherige Annahme ist, dass die Daten die Blöcke perfekt füllen und sich nicht ändern. Wenn jedoch Änderungen im Datenblock passieren, verändern sich die Indizes wie folgt:

Overflow Blocks werden in dünnbesetzten Indizes beispielsweise nicht repräsentiert. Neue Blöcke in der Sequenz hingegen benötigen einen neuen Indexeintrag. Diese Indexänderung birgt dieselben Probleme wie einen Datenänderung: Eine neuplatzierung der Blöcke und Indizes mit höheren Stufen.
Werden Tupel verschoben, muss der Index nur angepasst werden.
     
Generell können Indizes wie normale data files behandelt werden und es können dieselben Strategien angewendet werden.

**Änderungsoperationen Beispiele**

<img src="pictures/Änderungsoperationen_0.png" alt="Änderungsoperationen_0" width="500" style="background-color: white;"/>

**Beispiel 1: Dichtbesetzter Index**
<br>
Der Datensatz mit K = 30 wird gelöscht. Wir nehmen an der Block kann/soll **nicht** reorganisiert werden und an der gelöschten Stell wird ein Grabstein gesetzt. Der Datensatz 40 wird nicht verschoben und der Index wird reorganisiert werden, wenn er sich im Hauptspeicher befindet.

<img src="pictures/Änderungsoperationen.png" alt="Änderungsoperationen" width="500" style="background-color: white;"/>
   

**Beispiel 2: Dünnbesetzter Index**
<br>
Der Datensatz mit K = 30 wird gelöscht.  Wir nehmen an der Block kann/soll reorganisiert werden. Der Datensatz 40 wird nach dem Löschen der 30 verschoben und der Index wird aktualisiert. Wenn nun auch Datensatz mit K=40 gelöscht werden soll entsteht ein leerer Block und die 50 und 70 schieben sich nach oben.

<img src="pictures/Änderungsoperationen_2.png" alt="Änderungsoperationen_2" width="500" style="background-color: white;"/>

**Beispiel 3: Dünnbesetzter Index**  
<br>
Einfügen eines Datensatzes 15: Die 15 müsste zwischen die 10 und 20, aber Block 1 ist voll. Demnach wird der Datensatz 20 in den nächsten Block verschoben und der Block wird reorganisiert. Der Datensatz 15 wird eingefügt und der Index wird aktualisiert.

<img src="pictures/Änderungsoperationen_3.png" alt="Änderungsoperationen_3" width="500" style="background-color: white;"/>

**Beispiel 4: Dünnbesetzter Index**

Eine weitere Möglichkeit die 15 einzufügen ist mit einem Overflow Block. Der Block 1 ist weiterhin voll. Der Datensatz 20 wird in einen Overflow Block verschoben und der Datensatz 15 wird eingefügt. Der Index bleibt in diesem Fall gleich. 

<img src="pictures/Änderungsoperationen_4.png" alt="Änderungsoperationen_4" width="500" style="background-color: white;"/>

## Sekundärindizes auf nichtsequenziellen Dateien

Zuvor haben wir uns mit Primärindizes, also sortierten Daten, beschäftigt. Oft ist es aber sinnvoll, mehrere Indizes pro Relation zu haben. Damit kommen wir zu nichtsequentiellen Dateien, also unsortierten Daten. Im folgenden Beispiel haben wir die Relation *Schauspieler*, in welcher der Name der Primärschlüssel ist. Suchen wir nun das Geburtsdatum, müssen wir von Block zu Block springen und können keine Annahme über nachfolgende Blöcke treffen. 

```
SELECT Name, Adresse
FROM Schauspieler
WHERE Geburtstag = DATE '1952-01-01'
```

Nutzen wir nun einen Sekundärindex auf Geburtstag, beschleunigen wir die Anfragebearbeitung. Dieser Index kann nur im Nachhinein erstellt werde, da die Daten bereits vorliegen, anders als beim Primärindex, bei dem man im Optimalfall die Tupel während des Einfügens schon sortieren kann. 

``` CREATE INDEX GEB_IDX ON Schauspieler(Geburtstag)```

Sekundärindizes bestimmen nicht die Platzierung der Datensätze, sondern geben nur den Speicherort an. Deshalb sind dünnbesetzte Indizes sinnlos, da wir beispielsweise nicht wissen, ob die gleichen Geburtstage innerhalb des gleichen Blocks liegen oder auf vielen verschiedenen Blöcken verteilt sind.

### Aufbau von Sekundärindizes
     
Wie zuvor erwähnt sind Sekundärindizes dichtbesetzt. Die Schlüssel sind dabei schon sortiert und es können Duplikate vorkommen. Ein Index der zweiten Stufe wäre wiederum dünnbesetzt. Die Suche hat in der Regel hohe I/O-Kosten. Suchen wir beispielsweise die *20* müssen fünf Blöcke gelesen werden. Das lässt sich auch leider nicht ändern, da die Daten nach einem anderen Schlüssel sortiert sind.

<img src="pictures/Aufbau-Sekundärindizes.png" alt="Aufbau-Sekundärindizes" width="500" style="background-color: white;"/>

### Anwedungen

Es gibt Anfragen, die sich auf mehrere Attribute beziehen. Dann braucht man mehrere Indizes. Liegen die Daten nicht sortiert vor, nutzt man Sekundärindizes. Diese verbrauchen zwar mehr Speicher, aber wenigstens weiß man dann, wo was liegt. Weiß man, dass Fremdschlüsselbeziehungen existieren, kann man Informationen von zwei Relationen zusammen in sogennanten *Clustered files* speichern. Das ist besonders effizient, wenn es in einer Anfrage viele Join Operationen gibt.
  
**Anwendung: Clustered file**

- Filme(Titel, Jahr, Länge, inFarbe, Studioname, Produzent)
- Studio(Name, Adresse, Präsident)

Schauen wir uns dazu ein Beispiel an. Wir haben die beiden Relationen *Filme* und *Studio*. Studioname ist der Fremdschlüssel zu Name aus der Relation Studio. Häufige Anfragen können wir folgt aussehen:

```
SELECT Titel, Jahr
FROM Filme, Studio
WHERE Filme.Studioname = Studio.Name
AND Präsident = ?
```

Abgespeichert wird immer zuerst ein Studio und dann alle Filme aus diesem Studio.

<img src="pictures/cluster-file.png" alt="cluster-file" width="500" style="background-color: white;"/>

1. Index auf Präsident findet schnell Studio-Datensatz
2. Entsprechende Filme folgen direkt <br>

Anfragen direkt nach Filmen benötigen ebenfalls einen Sekundärindex. Ist beispielsweise der Titel der Primärschlüssel, muss doppelt gespeichert werden, damit die Daten sortiert vorliegen. Da es aber so ist, dass alle Filme jedes Studios jeweils daneben liegen, braucht man einen Sekundärindex, um auch den Primärschlüssel von *Filme* zu indizieren. 


### Indirektion für Sekundärindizes

Bisher haben wir uns clustered files angeschaut und Relationen, bei denen mehrere Attribute indiziert wurden. Dabei kann es leicht passieren, dass Dinge mehrfach indiziert und gespeichert werden. Hier kann die *Indirektion* Abhilfe schaffen.

Für Nicht-Primärschlüssel kann ein Index sehr groß sein. Aus dem Grund kann man unbestimmte Buckets dazu nehmen, die dann als abstrakte Schlüssel für die entsprechenden Datenpunkte gelten. In der Abbildung unten sieht man, dass die *10* auf so ein Bucket zeigt, in welchem drei Schlüsselpaare sind, die wiederum auf die entsprechenden Werte zeigen.

Dies spart Platz, falls die Suchschlüssel größer sind als der Bucketeintrag und im Durchschnitt mindestens zweimal auftauchen. Ein weiterer Vorteil ist, dass bestimmte Anfragen direkt anhand der Buckets beantwortet werden können. 
Außerdem können wir bei mehreren Selektionsbedigungen mit jeweils einem Sekundärindex, wie bei der Anfrage unten, die Schnittmenge der Pointer in den Buckets bilden. Hier haben wir jeweils einen Sekundärindex auf *StudioName* und *Jahr*. Während die Pointer der Indexeinträge für *StudioName* auf Buckets zeigen, können parallel die Pointer für *Jahr* auf dieselben zeigen, so dass man sofort weiß, welche Blöcke gelesen werden müssen. 

Filme(Titel, Jahr, Länge, inFarbe, Studioname, Produzent)

```
SELECT Titel FROM Filme
WHERE StudioName = 'Disney'
AND Jahr = 1995
```

<br>

<img src="pictures/Indirektion-Sekundärindizes.png" alt="Indirektion-Sekunderindizes" width="500" style="background-color: white;"/>


**Indirektion im Alltag**

Hier ein weiteres Beispiel für die Indirektion. Sucht man hier ein Wort, stehen die Buckets, in dem Fall die Seitenzahlen, direkt dahinter. Diese Indirektion ist speziell, denn sie hat eine bestimmte Indexstruktur und zwar *Invertierte Indizes*. Das schauen wir uns im folgenden Absatz genauer an.

<img src="pictures/Indirektion-Alltag.png" alt="Indirektion-Alltag" width="500" style="background-color: white;"/>

### Indexierung mittels invertierter Dateien

Invertierte Dateien oder auch invertierte Indizes speichern für jeden Term eine Liste mit Verweisen zu allen Dokumenten oder Dateien, in denen dieser Term vorkommt. Verweise sind aus konzeptioneller Sicht eine ID, die dem jeweiligen Dokument oder der Datei zugeordnet ist. 

Bei der Suche nach Dokumenten, die mehrere Terme oder eine bestimmte Konstellation von Termen enthalten sollen, z.B" “Finde alle Dokumente, die zwar Informationen über Golf enthalten aber nicht über ",W” können die entsprechenden Dateilisten jedes Termes durch Mengenoperationen zusammengeführt werden. In diesem Beispiel müsste man alle I s a diei" “"W” verweenst von allen IDs auf di" “Go"f” verweist abziehen. Boolesche Operationen wie AND, OR und NOT können hier in linearer Zeit durchgeführt werden, da die Einträge aufsteigend sortiert aufgelistet sind und ermöglichen komplexe Suchanfragen.

In Abbildung 1 sehen die von Dokumenten-für auf den enthaltenen Text. Jede Zeile steht für eine Daei/ ein Dokument. Die invertierte Datei sehen wir in Abbildun.




**Abbildung 1**


<img src="pictures/Abbildung1.png" alt="Abbildung1" width="300" style="background-color: white;"/>

<br>

**Abbildung 2**

<img src="pictures/Abbildung2.png" alt="Abbildung2" width="300" style="background-color: white;"/>


$\tiny{Quelle:\ Nagy \ Istvan \ (2001): Indexierung \ mittels \ invertierter \ Dateien. https://www.cis.uni-muenchen.de/people/Schulz/SeminarSoSe2001IR/Nagy/node4.html\#bsp_text }$

Invertierte Indizes finden ins besondere in Dokumentendatenbanken Anwendung. Für die weiteren Inhalte der Vorlesung spielen sie keine Rolle. Das sollte als Beispiel für die weitere Anwendungen der Indirektion dienen. 

### B-Bäume

####  Allgemein

Bisher haben wir zweistufige Indizes zur Beschleunigung des Zugriffs betrachtet. Jetzt befassen wir uns mit mehrstufige Indizes. 

B-Bäume haben so viele Stufen wie nötig. Die Blöcke sind immer mindestens zur Hälfte gefüllt und Overflow blocks sind nicht notwendig. 
B-Bäume sind außerdem balanciert, d.h. jeder Weg von der Wurzel bis zum Blatt ist gleich lang. 

<img src="pictures/B-Bäume.png" alt="B-Bäume" width="500" style="background-color: white;"/>


#### Struktur

Bei B-Bäumen handelt es sich um Index-Blöcke die in einem Baum organisiert sind. Dieser ist balanciert und hat einen Parameter *n*. Jeder Block enthält bis zu *n* Suchschlüssel und bis zu *n + 1* Pointer, also ein Pointer mehr als die Indexblöcke zuvor. *n* wird entsprechend der gegebenen Blockgröße so groß wie möglich gewählt. Bei beispielsweise 4096 Byte pro Block, 4 Byte pro Schlüssel und 8 Byte pro Pointer muss unser *n* = 340 sein:

4 n + 8(n+1) ≤ 4096 => n = 340

Bei einer Höhe von 3 lassen sich damit 340 ∙ 341 ∙ 341 = 39.535.540 Schlüssel indizieren.

#### Einfügen in B-Bäume – Beispiel

<img src="pictures/Einfügen-in-B-Bäume.png" alt="Einfügen-in-B-Bäume" width="500" style="background-color: white;"/>

#### Hier B+ Baum**

Bei B+ Bäumen sind die Schlüssel in den Blättern die Schlüssel aus den Daten, sortiert über alle Blätter verteilt (von links nach rechts). 
Die Pointer der Wurzel zeigen auf einen B-Baum Block in der Ebene darunter. Mindestens zwei Pointer werden verwendet. Von den Blättern zeigt der letzte Pointer auf das nächste rechte Blatt, während die übrigen Pointern auf mindestens $ \lfloor \frac {(n+1)}{2} \rfloor$ Datenblöcke zeigen. Die Pointer der inneren Knoten zeigen auch auf die B-Baum Blöcke von darunterliegenden Ebenen. Dabei werden mindestens $ \lceil \frac {(n+1)}{2} \rceil $ verwendet. 

Falls *j* Pointer verwendet werden, gibt es *j-1* Schlüssel in einem Block. Der erste Pointer zeigt auf den Teilbaum mit den Schlüsselwerten < K1 während der zweite Pointer auf den Teilbaum mit den Schlüsselwerten ≥ K1 und < K2 zeigt, usw.
     
#### Rechenbeispiele

Schauen wir uns das an ein paar Beispielen genauer an. Ist unser *n* = 3 haben alle Knoten maximal 3 Suchschlüssel und 4 Pointer. Die Wurzel hat mindestens 1 Suchschlüssel und 2 Pointer, während die Blätter mindestens $ \lfloor \frac {(3+1)}{2} \rfloor + 1 $ = 3 Pointer (plus 1 weil der letzte Pointer auf das nächste Blatt zeigt) und 2 Suchschlüssel haben. Die inneren Knoten haben mindestens $ \lceil \frac {(3+1)}{2} \rceil $ = 2 Pointer und 1 Suchschüssel.

<img src="pictures/Rechenbeispiele.png" alt="Rechenbeispiele" width="500" style="background-color: white;"/>

Analog für n = 4 und n = 5:

n = 4
- Alle Knoten: Maximal 4 Suchschlüssel und 5 Pointer
- Wurzel: Mindestens 1 Suchschlüssel und 2 Pointer
- Innere Knoten: Mindestens $ \lceil \frac {(n+1)}{2} \rceil $ Pointer = 3 Pointer
     - => Mindestens 2 Suchschlüssel
- Blätter:
     - 1 Pointer zum nächsten Blatt + mindestens $ \lfloor \frac {(n+1)}{2} \rfloor$ weitere Pointer = 3 Pointer
     - => Mindestens 2 Suchschlüssel
<br>

n = 5
- Alle Knoten: Maximal 5 Suchschlüssel und 6 Pointer
- Wurzel: Mindestens 1 Suchschlüssel und 2 Pointer
- Innere Knoten: Mindestens $ \lceil \frac {(n+1)}{2} \rceil $ Pointer = 3 Pointer
    - => Mindestens 2 Suchschlüssel
- Blätter:
     - 1 Pointer zum nächsten Blatt + mindestens $ \lfloor \frac {(n+1)}{2} \rfloor$ weitere Pointer = 4 Pointer
     - => Mindestens 3 Suchschlüssel



#### Alternative Definition

In der Vorlesung verwenden wir den Parameter *n*. In Lehrbüchern wird alternativ aber auch der Parameter *k* verwendet. Anders als bei der Variante mit *n* hat ein Block mindestens *k* und höchstens *2k* Suchschlüssel. *x + 1* Pointer pro Block bleiben aber. Genauso wie die Tatsache, dass ein innerer Block immer einen Pointer mehr hat als Suchschlüssel und ein Blatt genauso viele Pointer wie Suchschlüssel.


#### Beispiel Blattknoten

Hier werden wir ein paar Beispiele für Blattknoten mit *n = 3* sehen. Das bedeutet, es gibt maximal 3 Schlüssel und 4 Pointer.

**Volles Blatt**

Hier sehen wir ein volles Blatt. Wir haben die 3 Suchschlüssel 57, 81 und 95 sowie 4 Pointer. Der erste Pointer zeigt zu dem Block mit dem Suchschlüssel 57. Der zweite zu dem Block mit dem Suchschlüssel 81 und der dritte zum Block mit dem Suchschlüssel 95. Der letzte Pointer zeigt auf das nächste Blatt.

<img src="pictures/Beispiel-Blattknoten.png" alt="Beispiel-Blattknoten" width="500" style="background-color: white;"/>

**Teilweise gefülltes Blatt**

Nun sehen wir ein teilweise gefülltes Blatt. Hier haben wir nur 2 Suchschlüssel, 57 und 81, und 3 Pointer. Die ersten beiden zeigen wieder auf Blöcke mit den Schlüsseln 57 und 81 während der letzte wieder auf das nächste Blatt zeigt. Ein Blatt mit nur einem Suchschlüssel ist nicht erlaubt, da wir mindestens $ \lfloor \frac {(3+1)}{2} \rfloor + 1 $ = 3 Pointer und damit 2 Suchschlüssel benötigen. 

<img src="pictures/Beispiel-Blattknoten_2.png" alt="Beispiel-Blattknoten_2" width="500" style="background-color: white;"/>

#### Beispiel innerer Knoten

Schauen wir uns das gleiche auch für innere Knoten an.

**Voller innerer Knoten**

Hier haben wir einen vollen inneren Knoten mit 3 Suchschlüsseln und 4 Pointern. Der erste Pointer zeigt auf die Schlüssel die kleiner sind als 57, der zweite Pointer auf die Schlüssel die größer oder gleich 57 sind **und** kleiner als 81. Der dritte Pointer zeigt auf die Schlüssel die größer oder gleich 81 **und** kleiner 95 sind und der letzte Pointer zeigt auf Suchschlüssel größer oder gleich 95. 

<img src="pictures/Beispiel-Innerer-Knoten.png" alt="Beispiel-Innerer-Knoten" width="500" style="background-color: white;"/>

**Teilweise gefüllter innerer Knoten**

Bei diesem teilweise gefüllten inneren Knoten haben wir nur einen Suchschlüssel und 2 Pointer. Diese zeigen wiederum auf Suchschlüssel die kleiner als 57 und größer oder gleich 57 sind. Keine Suchschlüssel in einem Knoten zu haben ist nicht erlaubt, da wir mindestens $ \lceil \frac {(3+1)}{2} \rceil $ = 2 Pointer und 1 Suchschüssel benötigen.

<img src="pictures/Beispiel-Innerer-Knoten_2.png" alt="Beispiel-Innerer-Knoten_2" width="500" style="background-color: white;"/>

#### Beispiel B-Baum

In dem folgenden Beispiel handelt es sich um einen dichtbesetzten B-Baum mit *n = 3*, 2 Pointern und einem Suchschlüssel. Dichtbesetzt, das heißt, dass in den Blättern jeder Schlüssel genau einmal sortiert auftaucht. 

Schauen wir uns ein paar Knoten genauer an. Mit 13 in der Wurzel gehen alle Zahlen die kleiner sind, auf die linke Seite und alle die größer oder gleich 13 sind auf die rechte. Im darunterliegenden linken Knoten haben wir den Schlüssel 7. Alle Elemente kleiner als 7 gehen in das linke Blatt, alle größer oder gleich 7 in das rechte. Jedes Blatt hat Pointer, die auf Datenblöcke zeigen und einen Pointer, der auf das nächste Blatt zeigt.

<img src="pictures/Beispiel-B-Baum.png" alt="Beispiel-B-Baum" width="500" style="background-color: white;"/>


#### Anwendungen von B-Bäumen

B-Bäume können verschiedene Index-Rollen übernehmen. Ist der Suchschlüssel gleichzeitig auch der Primärschlüssel, können wir dichtbesetzte Indizes verwenden. Hierbei spielt es keine Rolle, ob die data files sortiert sind oder nicht. Bei dünnbesetzten Indizes sind die date files immer sortiert. Sollte das aber mal doch nicht der Fall sein, müssen wir Indirektion verwenden. 
Sollte der Suchschlüssel nicht der Primärschlüssel sein, müssen die data files nach dem Suchschlüssel sortiert sein. Außerdem müssen die Pointer immer auf den jeweils ersten Wert zeigen. Dies schauen wir uns im folgenden Abschnitt genauer an. 
   
#### B-Bäume auf nicht-Primärschlüsseln

Haben wir B-Bäume auf nicht-Primärschlüsseln, ändert sich die Bedeutung der Pointer auf den inneren Ebenen, da es jetzt vorkommen kann, dass wir Duplikate in unseren Blättern haben. 

Gegeben sind wieder die Schlüssel K $_{1}$ bis K $_{j}$. K $_{i}$ ist hierbei der kleinste *neue* Schlüsselwert, der vom (j+1)-ten Pointer erreichbar ist. Das bedeutet, es gibt keinen Schlüssel K $_{i}$ im linken Teilbaum, aber mindestens ein Vorkommen des Schlüsselwertes im Teilbaum vom (j+1)-ten Pointer. Das Problem ist: Es gibt nicht immer einen solchen Schlüssel. 

<img src="pictures/B-Bäume-auf-nicht-Primärschlüsseln.png" alt="B-Bäume-auf-nicht-Primärschlüsseln" width="500" style="background-color: white;"/>
<img src="pictures/B-Bäume-auf-nicht-Primärschlüsseln_2.png" alt="B-Bäume-auf-nicht-Primärschlüsseln_2" width="500" style="background-color: white;"/>

### B-Bäume Suche

#### Allgemein

Wir nehmen wieder an, dass unser Suchschlüssel ein Primärschlüssel ist. Jeder Schlüssel ist dann dementsprechend nur einmal in einem Blatt vorhanden, es kommen also keine Duplikaten vor.

Gesucht ist wieder ein Schlüssel *K*. Wir fangen bei einem Blattknoten an und schauen, ob der Schlüssel dort vorkommt oder nicht. Falls nicht, suchen wir nach einem Paar von Schlüsseln, zwischen denen unser Schlüssel vorkommen könnte und geben den Pointer weiter.

Im folgenden schauen wir uns zwei Beispiele dazu an. 

#### Beispiel Suche im B-Baum

Suchen wir beispielsweise nach dem Datensatz 40, gehen wir erstmal in den Wurzelknoten. 13 ist hier der einzige Wert. Da 40 größer ist, springen wir zu dem rechten inneren Knoten und können dann mit binärer Suche vergleichen, wo wir weitersuchen müssen. Folgen wir dann dem Pointer zwischen 31 und 43 und suchen in dem Blattknoten, finden wir den Datensatz nicht und sind fertig. 

<img src="pictures/Beispiel-Suche-im-B-Baum.png" alt="Beispiel-Suche-im-B-Baum" width="500" style="background-color: white;"/>

Hier das gleiche Prinzip. 13 ist kleiner als 37, also springen wir auch hier zum rechten inneren Knoten. Wir können wieder dem Pointer zwischen 31 und 43 folgen und finden dann die 37.

<img src="pictures/Beispiel-Suche-im-B-Baum_2.png" alt="Beispiel-Suche-im-B-Baum_2" width="500" style="background-color: white;"/>

#### Bereichsanfragen (range queries)

Anfragen mit Ungleichheit in WHERE-Klausel
```
SELECT * FROM R
WHERE R.k > 40
```

```
SELECT * FROM R
WHERE R.k >= 10 AND R.k <= 25
``` 

Mit B-Bäumen lassen sich Ungleichheiten, also die Anfrage über bestimmte Bereiche von Werten, effizient behandeln. 

Suchen wir zum Beispiel in dem Bereich [a,b], suchen wir zunächst einen Blattknoten der a enthält. Die Suche führen wir solange weiter, bis wir bei b ankommen. Dann können wir unsere Suche abschließen. Haben wir keinen B-Baum, muss die Suche eventuell intensiviert und die Liste vorher sortiert werden. 

Bei offenen Bereichen, wie [-∞ , b] bzw. [a, ∞] ist Suche ähnlich.

Im folgenden ein Beispiel: Wir suchen alle Schlüssel zwischen 10 und 25. Zunächst suchen wir die 10 und gehen links runter in den Block mit der 7 und der 11. Von da schauen wir uns die Blöcke rechts an, bis wir den Block mit der 23 und der 29 erreichen. Da die 29 größer als 25 ist, ist unsere Suche hier beendet. 

<img src="pictures/Bereichsanfragen.png" alt="Bereichsanfragen" width="500" style="background-color: white;"/>

Schauen wir uns nun an, wie man B-Baume updated.

### B-Bäume Updates

#### Einfügen in B-Bäume

Möchte man etwas in einen B-Baum einfügen, geht man rekursiv vor. Zunächst suchen wir das entsprechende Blatt. Falls dort Platz ist, fügen wir den Schlüssel und den Pointer ein. Falls nicht, kommt es zum *Überlauf*. Dabei wird das Blatt in zwei Teile gespalten und die Schlüssel gleichmäßig auf diese verteilt. Durch diesen "split" ist es erforderlich, dass Schlüssel/Pointer-Paare im Elternknoten eingefügt, also rekursiv nach oben geschoben, werden. Sollte die Wurzel voll sein, muss auch diese gesplittet und eine neue mit nur einem Schlüssel erzeugt werden.


Im folgenden Beispiel soll die 40 eingefügt werden. Wir sehen, dass das entsprechende Blatt voll ist, also muss dieses gesplittet werden. 
<br>

<img src="pictures/Einfügen-in-B-Bäume-Beispiel.png" alt="Einfügen-in-B-Bäume-Beispiel" width="500" style="background-color: white;"/>

<br>
Nun müssen wir uns fragen, welcher Schlüssel nach oben wandert. Wir nehmen idealerweise das erste Element in einem neuen Block, hier die 40. Diese wandert dann nach oben, sodass wir den Elternknoten auch noch splitten müssen.
<br>

<img src="pictures/Einfügen-in-B-Bäume-Beispiel_2.png" alt="Einfügen-in-B-Bäume-Beispiel_2" width="500" style="background-color: white;"/>

<br>
Wir brauchen nun ein neues Schlüssel/Pointer-Paar. Da die 40 auch hier das erste Element im neuen Block ist, würde es sich anbieten diese in die Wurzel zu ziehen. 

<img src="pictures/Einfügen-in-B-Bäume-Beispiel_3.png" alt="Einfügen-in-B-Bäume-Beispiel_3" width="500" style="background-color: white;"/>

<br>

Dadurch müssten wir aber zu viel umstrukturieren, deshalb können wir auch die 31 aus dem linken Block nehmen.
<br>

<img src="pictures/Einfügen-in-B-Bäume-Beispiel_4.png" alt="Einfügen-in-B-Bäume-Beispiel_4" width="500" style="background-color: white;"/>

**Kosten für Einfügen**
       
Die Kosten für das Einfügen in einen B-Baum sind abhängig von der Höhe *h* des Baumes. Im Rahmen der Vorlesung haben wir meist B-Bäume der Höhe 3. 
Die Suche nach einem Blattknoten kostet *h*. Ist keine Teilung notwendig, muss man nur *h* Blöcke lesen und 1 Index-Block schreiben. Damit liegen die Gesamtkosten bei *h + 1*. Falls doch eine Teilung notwendig ist, muss man im schlimmsten Fall bis nach oben zu der Wurzel. Da hilft auch Caching nicht, da die Knoten geschrieben werden müssen. Insgesamt kommt man auf *3 h + 1*, da man auf jeder Ebene suchen, teilen und gegebenenfalls Überlauf behandeln und zusätzlich eine neue Wurzel generieren muss.  

#### Löschen aus B-Bäumen

Beim löschen auf einem B-Baum wird der entsprechende Knoten gesucht und der Schlüssel dann gelöscht. Befindet sich dann immer noch die minimale Menge an Schlüsseln im Knoten ist alles gut und wir müssen nichts tun. Sind aber zu wenig Schlüssel im Knoten, müssen wir *mergen*, d.h. wir können uns aus den Geschwisterknoten links oder rechts die mehr als die minimale Schlüsselmenge haben, einen Schlüssel "klauen". Dabei müssen auch (fast) immer die Elternknoten angepasst werden. Ist das mergen aber nicht möglich, da die Geschwisterknoten nur die minimale oder sub-minimale Schlüsselmenge besitzen, kann man diese aber vereinen. Auch hier müssen die Elternknoten dann angepasst oder ggf. rekursiv nach oben gelöscht werden.


Schauen wir uns dazu ein Beispiel an. Es soll der Datensatz mit dem Schlüssel 7 gelöscht werden. 

<img src="pictures/Löschen-aus-B-Bäumen-Beispiel.png" alt="Löschen-aus-B-Bäumen-Beispiel" width="500" style="background-color: white;"/>

Ohne die 7 sind zu wenig Schlüssel im Knoten, also nehmen wir uns aus dem linken Geschwisterknoten die 5 dazu und passen dann noch den Elternknoten an.

<img src="pictures/Löschen-aus-B-Bäumen-Beispiel_2.png" alt="Löschen-aus-B-Bäumen-Beispiel_2" width="500" style="background-color: white;"/>

Jetzt soll die 11 entfernt werden. 

<img src="pictures/Löschen-aus-B-Bäumen-Beispiel_3.png" alt="Löschen-aus-B-Bäumen-Beispiel_3" width="500" style="background-color: white;"/>

Auch hier sind jetzt zu wenig Schlüssel im Knoten. In dem Fall können die Geschwisterknoten aber nichts mehr abgeben, also mergen wir mit dem linken Geschwisterknoten.

<img src="pictures/Löschen-aus-B-Bäumen-Beispiel_4.png" alt="Löschen-aus-B-Bäumen-Beispiel_4" width="500" style="background-color: white;"/>

Den leeren Knoten können wir löschen und müssen dann noch den Elterknoten anpassen, da dieser zu klein ist. Die 5 wird da aber nicht mehr gebraucht, also kann diese auch entfernt werden.

<img src="pictures/Löschen-aus-B-Bäumen-Beispiel_5.png" alt="Löschen-aus-B-Bäumen-Beispiel_5" width="500" style="background-color: white;"/>

Der zweite Pointer des jetzt leeren Knoten zeigt nun auf den rechten Knoten. Jetzt passt es aber nicht mehr mit der 13 in der Wurzel, deshalb müssen wir hier auch noch eine Anpassung vornehmen.
 
<img src="pictures/Löschen-aus-B-Bäumen-Beispiel_6.png" alt="Löschen-aus-B-Bäumen-Beispiel_6" width="500" style="background-color: white;"/>

Wir ziehen die 13 aus der Wurzel runter und die 23 dafür hoch.

<img src="pictures/Löschen-aus-B-Bäumen-Beispiel_7.png" alt="Löschen-aus-B-Bäumen-Beispiel_7" width="500" style="background-color: white;"/>

**Kosten für Löschen**

- Suche und lokales Löschen: h + 1
     - Schreibe Blattknoten
- Bei merge mit Geschwisterknoten: h + 5
     - Prüfe rechts und links
     - Schreibe Block und veränderten Nachbarn
     - Schreibe Elternknoten
- Bei merge bis zur Wurzel: 3h - 2

**Löschen aus B-Bäumen?**
- Annahme: Tendenziell wachsen Datenmengen
- Folgerung: Nie Knoten des B-Baum löschen
    - Knoten, die durch Löschen zu klein werden, werden früher oder später wieder gefüllt.
    - Grabstein auf Datenblock genügt, B-Baum muss nicht geändert werden

### Effizienz von B-Bäumen

Suche, Einfügen und Löschen sollen möglichst wenig I/O-Operationen benötigen.
- Je größer n gewählt wird, desto seltener müssen Blöcke verschmolzen oder getrennt werden.
    - Meist auf Blattknoten beschränkt
- Suche (best case)
    - Anzahl I/Os entspricht der Höhe des Baums
    - + 1 I/O auf den Daten für Lesen
    - oder + 3 I/O auf den Daten für Einfügen oder Löschen
- Typische Höhe eines B-Baums: 3
    - 340 Schlüssel-Pointer-Paare pro Block
    - Annahme: Füllstand durchschnittlich 255
    - => 255 innere Knoten = > 255² = 65025 Blätter = 255³ Pointer
    - Insgesamt über 16 Mio. Datensätze
    - Maximal: 340³ = 39 Mio. Datensätze
- Zugriff mit 2 I/Os: Nur Wurzel im Hauptspeicher
- Zugriff mit 1 I/O: Wurzel und 255 innere Knoten im Hauptspeicher

#### Bulk-loading

- Einfügevarianten
    - Einfügen von Daten in Relation, die bereits mit existierendem B-Baum indiziert ist.
    - Erstellung eines neuen B-Baums auf existierenden Daten
        - Sukzessives Einfügen jedes Datensatzes ist ineffizient.
        - Immer wieder über die Wurzel suchen
        - Viel I/O für Indexblöcke, die nicht im Hauptspeicher sind.
- Besser: Vorsortierung
    - Schritt 1: Erzeuge Schlüssel-Pointer Paare für alle Blöcke
    - Schritt 2: Sortierung der Paare nach Suchschlüssel
    - Schritt 3: Füge nun sukzessive die Paare ein
    
**Beispiel**
- n = 3
- Daten: 2; 3; 5; 7; 11; 13; 17; 19; 23; 29; 31; 37; 41; 43; 47
- Erzeuge halbgefüllte Blattknoten

<img src="pictures/bulk-loading-Beispiel.png" alt="bulk-loading-Beispiel" width="500" style="background-color: white;"/>

- Erzeuge Wurzel für erste beiden Blattknoten

<img src="pictures/bulk-loading-Beispiel_2.png" alt="bulk-loading-Beispiel_2" width="500" style="background-color: white;"/>

- Füge nächstes Blatt ein

<img src="pictures/bulk-loading-Beispiel_3.png" alt="bulk-loading-Beispiel_3" width="500" style="background-color: white;"/>

- Füge nächstes Blatt ein

<img src="pictures/bulk-loading-Beispiel_4.png" alt="bulk-loading-Beispiel_4" width="500" style="background-color: white;"/>

- Füge nächstes Blatt ein

<img src="pictures/bulk-loading-Beispiel_5.png" alt="bulk-loading-Beispiel_5" width="500" style="background-color: white;"/>

- Füge nächstes Blatt ein

<img src="pictures/bulk-loading-Beispiel_6.png" alt="bulk-loading-Beispiel_6" width="500" style="background-color: white;"/>

- Füge letztes Blatt ein

<img src="pictures/bulk-loading-Beispiel_7.png" alt="bulk-loading-Beispiel_7" width="500" style="background-color: white;"/>

**Bulk-Loading mit hohem Füllstand**
- Schritt 1: Erzeuge Schlüssel-Pointer Paare für alle Blöcke
- Schritt 2: Sortierung der Paare nach Suchschlüssel
- Schritt 3: Erzeuge volle B-Baum Blätter
- Schritt 4: Konstruiere innere Knoten aufgrund der Blätter
- Ergebnis: Perfekter Füllstand
- Anwendungen:
     - Read-only Relationen
     - Append-only Relationen

### B-Baum Varianten: B-Baum (ohne „+“)

- Bisher: B+ Baum
    - Pointer auf Datensätze nur in Blattknoten
- B-Baum
    - Pointer auf Datensätze in allen Knoten
    
- Vorteil: Durchschnittlich schnellere Suche als in B+-Bäumen
- Nachteil: Blätter und innere Knoten haben unterschiedliche Struktur
    - Verwaltung schwieriger
- Nachteil: Weniger bushy; Platz wird für Pointer auf Datensätze „verschwendet“
    - Dadurch größere Höhe
- Nachteil: Löschen ist komplizierter
    - Löschung kann auch in inneren Knoten geschehen.
    - Schlüssel von einem Blatt muss gegebenenfalls nach oben wandern.

<img src="pictures/B-Baum-Varianten.png" alt="B-Baum-Varianten" width="500" style="background-color: white;"/>

**B*-Bäume**
- B*-Baum
    - Bei Überlauf werden nicht gleich zwei halb-leere Knoten erzeugt.
    - Stattdessen Neuverteilung über beide Nachbar-Blätter
    - Falls nicht möglich, erzeuge 3 neue Blätter aus 2 alten Blättern
    - Dadurch bessere Speicherausnutzung: Mindestens 66%
    
**Präfix-B+-Baum**
- Präfix-B+-Baum
    - Falls Suchschlüssel ein String ist => Hoher Speicherbedarf
    - Besser: Speichere nur „Trennwert“
    - Was trennt „Korn“ von „Licht“?
    - Am besten: Kleinster Trennwert
        - „L“
        - Präfix von „Licht“

<img src="pictures/B-Baum-Varianten_2.png" alt="B-Baum-Varianten_2" width="500" style="background-color: white;"/>
<img src="pictures/Bücher.png" alt="Bücher" width="500" style="background-color: white;"/>
<img src="pictures/Bücher_2.png" alt="Bücher_2" width="500" style="background-color: white;"/>

**B+-Bäume für BLOBs (und CLOBs)**
- Idee: Suchschlüssel repräsentiert Offsets im BLOB anstatt Suchschlüssel
    - Blätter zeigen auf Datenseiten des BLOBs
    - Positional B-Tree
 
<img src="pictures/B-Bäume_für_BLOBs.png" alt="B-Bäume_für_BLOBs" width="500" style="background-color: white;"/>

### Hashtabellen

#### Allgemeine Hash-Tabellen
**Hashtabellen – Grundprinzip**
- Hashfunktion
    - Input: Suchschlüssel K (Hash-Schlüssel)
    - Output: Integer zwischen 0 und B−1
        - B = Anzahl Buckets
    - Oft z.B. MOD(K/B) („Divisionsrestmethode“)
        - Bei Strings: Weise jedem Buchstaben Integer zu und summiere diese.
- Bucketarray
    - Array aus B Headern für B verkettete Listen

**Hashtabellen auf Festplatten**
- Bisher (im Studium): Hashtabellen im Hauptspeicher
- Nun: Blöcke statt Pointer auf Listen
    - Datensätze, die auf einen gemeinsamen Wert gehashed werden, werden in dem entsprechenden Block gespeichert.
    - Overflowblocks können ergänzt werden.
- Annahme: Zuordnung eines Hashwerts zur Speicheradresse eines Blocks möglich
    - Z.B.: Hashwert stellt Offset dar

<img src="pictures/Hashtabelle-auf-Festplatten.png" alt="Hashtabelle-auf-Festplatten" width="500" style="background-color: white;"/>

**Einfügen in Hashtabellen**
1. Berechne Hashwert
2. Falls Platz, füge Datensatz in entsprechenden Block ein
- Oder in einen Overflowblock mit Platz
3. Falls nicht, erzeuge neuen Overflowblock und füge dort ein
- Beispiel: Füge H ein
    - h(H) = 2
- Beispiel: Füge G ein
    - h(G) = 1

<img src="pictures/Einfügen-in-Hashtabelle.png" alt="Einfügen-in-Hashtabelle" width="500" style="background-color: white;"/>
    
**Löschen in Hashtabellen** 
1. Suche Bucket (+ Overflowblocks)
2. Suche Datensatz / Datensätze
3. Lösche sie
4. Gegebenenfalls Reorganisation und Entfernung von Overflowblocks
    - Gefahr der Oszillation
- Beispiel: Lösche C
    - G bewegen
 
<img src="pictures/Löschen-aus-Hashtabelle.png" alt="Löschen-aus-Hashtabelle" width="500" style="background-color: white;"/>

#### Vorschau: Consistent Hashing

- Problem of hash fragmentation:
    - Traditional hash functions require remapping of all tuples when a node goes offline
    - Because hash function must map to fewer nodes.
- Consistent hash function:
    - Minimizes the degree of remapping in case of the addition or removal of locations (nodes, slots)
    - Only K/n keys need to be remapped on average (K ... total number of keys, n ... number of slots)
    - Distributed Hash Tables (DHT)

<img src="pictures/consistent-hashing.png" alt="consistent-hashing" width="500" style="background-color: white;"/>
    
#### Effizienz statischer Hashtabellen

- Idealerweise: Pro Bucket nur ein Block 
    - Zugriffszeit lesen: 1 I/O 
    - Zugriffszeit Einfügen / Löschen: 2 I/O 
    - Viel besser als Dicht- oder Dünnbesetzte Indizes 
    - Besser als B-Bäume o Aber Nachteil: Bereichsanfragen nicht unterstützt o => Kein sequentielles Lesen von Disk 
- Weiteres Problem: B wird einmalig festgelegt 
    - Lange Listen von Overflowblöcken 
    - „Statische Hashtabellen“ 
- Lösung: „Dynamische Hashtabellen“ 
    - Hashtabellen können wachsen 
    - Ideal: B ≈ Anzahl Datensätze / Datensätze pro Block o => Ca. 1 Block pro Bucket

### Erweiterbare Hashtabellen

Neuerungen
- Indirektion
    - Bucket besteht aus Pointerarray statt Datenblock(s)
- Wachstum
    - Größe des Pointerarrays verdoppelt sich bei Bedarf
- Sharing
    - Buckets können sich Datenblöcke teilen
    - Wenn es passt
- Hashfunktion
    - Berechnet (zu) großes Bitarray (k bit; z.B. k = 32)
    - Bucketarray verwendet nur die ersten i Bits (i ≤ k)
    - => $2^i$ buckets
    
<br> <br> 

- k = 4

<img src="pictures/Erweiterbare-Hashtabellen.png" alt="Erweiterbare-Hashtabellen" width="500" style="background-color: white;"/>

#### Einfügen in erweiterbare Hashtabellen

- Ähnlich wie normale Hashtabelle
    - Berechne h(K); wähle die ersten i Bits
    - Suche Eintrag im Bucketarray und lade entsprechenden Block
- Falls Platz: Füge neuen Datensatz ein.
- Falls kein Platz und j < i (j ist aktuelle Bitanzahl des Blocks)
    - Spalte Block entzwei (split)
    - Verteile Datensätze gemäß des (j+1)ten Bits
        - Falls 0: Verbleib
        - Falls 1: Verschieben in neuen Block
    - Setze j+1 als neue Bitanzahl der beiden neuen Blöcke
    - Pointer im Bucketarray aktualisieren
    - Was wäre Pech?
        - Alle Datensätze landen wieder im gleichen Block. Dann wieder j++.
        
- Falls kein Platz und j = i (j ist aktuelle Bitanzahl des Buckets)
    - Global: i++ => Länge des Bucketarrays verdoppelt sich
    - Datenblöcke bleiben unverändert
    - Zwei neue Pointer zeigen zunächst auf gleichen alten Block
    - Dann: Spalte relevanten Block entzwei (split)
        - Weiter wie zuvor (denn nun j < i)

<img src="pictures/Einfügen-erweiterbare-Hashtabellen.png" alt="Einfügen-erweiterbare-Hashtabellen" width="500" style="background-color: white;"/>
<br> <br>
<img src="pictures/Einfügen-erweiterbare-Hashtabellen_2.png" alt="Einfügen-erweiterbare-Hashtabellen_2" width="500" style="background-color: white;"/>
<br> <br>
<img src="pictures/Einfügen-erweiterbare-Hashtabellen_3.png" alt="Einfügen-erweiterbare-Hashtabellen_3" width="500" style="background-color: white;"/>

#### Analyse erweiterbarer Hashtabellen

- Vorteile
    - Bei Suche: Nie mehr als ein Datenblock betrachten
        - Keine Overflow Blocks
    - Bucketarray eventl. im Hauptspeicher
- Nachteile
    - Bei Verdopplung und Split: Viel Arbeit
        - => Ab und zu dauert ein Einfügen sehr lang: Planbarkeit!
    - Bucketarray wächst schnell
        - Passt eventuell nicht mehr in Hauptspeicher
    - Platzverschwendung bei wenigen Datensätzen pro Block
        - Beispiel: 2 Datensätze pro Block
        - Datensatz 1: 00000000000000000001
        - Datensatz 2: 00000000000000000010
        - Datensatz 3: 00000000000000000011
        - => i = 20 (also 220 = 1Mio buckets)

### Lineare Hashtabellen

Ziel: Anzahl Buckets wachse nur langsam.
- Anzahl n der Buckets so gewählt, dass Datenblocks zu ca. 85% gefüllt sind.
- Overflow Blocks sind zugelassen.
    - Durchschnittliche Anzahl Overflow Blocks pro Block: <<1
- log2n Bits zur Identifizierung der Buckets
    - Wähle die jeweils letzten Bits des Hashwerts

<img src="pictures/Lineare-Hashtabellen.png" alt="Lineare-Hashtabellen" width="500" style="background-color: white;"/>
    
#### Lineare Hashtabellen – Einfügen

- Berechne h(K)
- Betrachte letzte i Bits; interpretiere als Integer m
- Falls m < n: Füge Datensatz in Bucket m ein.
- Falls m ≥ n:
    - => Bucket m existiert noch nicht
- Füge Datensatz in Bucket m–2i-1 ein.
    - D.h. erstes Bit des Schlüssels wird zu 0 (vorher 1).
- Falls kein Platz: Erzeuge Overflow Block
- Berechne r/n
    - Falls zu hoch (z.B. ≥ 1,7): Erzeuge einen neuen Bucket (n++)
    - Neuer Bucket hat nichts mit betroffenem Bucket zu tun.
- Falls nun n > 2i: i++
    - D.h. alle Bitsequenzen erhalten eine 0 am Anfang
    - Physisch ändert sich nichts
 
<img src="pictures/Einfügen-Lineare-Hashtabellen.png" alt="Einfügen-Lineare-Hashtabellen" width="500" style="background-color: white;"/> <br> <br>
<img src="pictures/Einfügen-Lineare-Hashtabellen_2.png" alt="Einfügen-Lineare-Hashtabellen_2" width="500" style="background-color: white;"/> <br> <br> <br>
<img src="pictures/Einfügen-Lineare-Hashtabellen_3.png" alt="Einfügen-Lineare-Hashtabellen_3" width="500" style="background-color: white;"/>

#### Hashing vs. B-Baum

- Hashing effizient zur Feststellung der Existenz eines Wertes
    - Overflow blocks nötig bei doppelten Schlüsseln
- B-Baum effizient für Bereichsanfragen und hält Daten sortiert vor
- Indexerzeugung in SQL:

```
CREATE [ UNIQUE ] INDEX indexname
ON table ( column [ ASC | DESC ]
[ , column [ ASC | DESC ] ... ] )
[CLUSTER]
[PCTFREE integer]
```

    - UNIQUE erlaubt NULL Werte
    - PCTFREE bestimmt Füllgrad
    - Keine Angabe über Art des Index
    - Keine Angabe über Parameter
    - Manchmal Hersteller-spezifische Syntax für Parameter